In [354]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
import util
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

import notebooks.util

%matplotlib

plt.close('all')

%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [355]:
prefix = 'eo_' + 'o3b_'

case, target = pickle.load(open('raw/%ssettings.pkl' % prefix, 'rb'))

import util
util.print_targets(case)

Targets per orbit
T_target: 960.00 Gb (min: 624.00 Gb, max: 1296.00 Gb)
L_target: 0.00 s
E_target: 41.16 kJ (max: -0.00 kJ)
P_target: 0.00 s


In [356]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/%spost_processed.h5" % prefix

row_limit = -1

hdf_store = pd.HDFStore(file_path)

instances_df = hdf_store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_df = instances_df.sort_index(0)

Performance plotting

In [357]:
plt.close('Performance')

# Plot the optimizations
fig = plt.figure(num='Performance', figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

N_orbits = case['T_sim_s'] / case['T_orbit_s']
scale_factors = np.array([1/-1e9, 1, 1/1e3, 1]) * \
                np.array([1/N_orbits, 1, 1/N_orbits, 1/case['T_sim_s']]) * \
                np.array([1/8, 1, 1, 100])

# problem, setting, res = pickle.load(open('raw/eo_o3b_nsga3_200_50_old.pkl', 'rb'))
# print("n_gen: %d, n_pop: %d" % (res.algorithm.n_gen, res.algorithm.pop_size))
# util.plot_performance_eo(axs, problem, setting, res, case, target, scale_factors)
# axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='NSGA-3 Golden')

problem, setting, res = pickle.load(open('raw/eo_o3b_nsga3_xover_300_300_75.pkl', 'rb'))
print("n_gen: %d, n_pop: %d" % (res.algorithm.n_gen, res.algorithm.pop_size))
util.plot_performance_eo(axs, problem, setting, res, case, target, scale_factors, plot_i=False)
axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='NSGA-3 Golden')

selection = 'raw/eo_o3b_nsga3_xover_300_300_75.pkl'

No handles with labels found to put in legend.


n_gen: 1000, n_pop: 300


In [358]:
points = []

# Targets
p = np.array([case['T_bitorbit_target'], case['L_sorbit_target'], case['E_Jorbit_target'], case['P_sorbit_target']/3])
p = p * np.array([-1, 1, 1, 1]) * np.array([N_orbits, 1, N_orbits, N_orbits])
p_fmt = p * scale_factors
points.append({'ind': -1, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'r', 'marker': 'x', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Closest to target
# problem, setting, res = pickle.load(open('raw/eo_o3b_nsga3_xover_300_300_150.pkl', 'rb'))
# ref = np.array([case['T_bitorbit_target'] * -1 * N_orbits, case['E_Jorbit_target'] * N_orbits, 0.0])
# #diff = np.abs(res.F[:, [0, 1, 2]] - ref) / (np.max(np.abs(res.F), axis=0) - np.min(np.abs(res.F), axis=0))
# diff = np.abs(res.F[:, [0, 1, 2]] - ref) / ref
# ind_close = np.argmin(np.linalg.norm(diff, axis=1))
# p = np.array(util.recompute_obj(problem, res, scale_factors=1))
# p = p[:, ind_close]
# p_fmt = p * scale_factors
# points.append({'ind': ind_close, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Select point
problem, setting, res = pickle.load(open(selection, 'rb'))
ind_close = 58
p = np.array(util.recompute_obj(problem, res, scale_factors=1))
p = p[:, ind_close]
p_fmt = p * scale_factors
points.append({'ind': ind_close, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

fig = plt.figure(num="Performance")
util.plot_points_eo(axs, points, scale_factors=scale_factors)
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%sresult.pdf' % prefix)

In [359]:
plt.close('Settings')

fig = plt.figure(num='Settings', figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

problem, setting, res = pickle.load(open(selection, 'rb'))
#problem, setting, res = pickle.load(open('raw/iot_telesat_rnsga2_1e-2_cont_300_75.pkl', 'rb'))
util.plot_settings(axs, problem, setting, res, points=points, scale_factors=scale_factors)

In [360]:
util.plot_power_energy(case, instances_df, problem, res.X[ind_close,:])
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%spower_energy.pdf' % prefix)
pass

bbox = plt.gca().get_position()

Max power: 53.74 W
Bandwidth: 300.00 MHz
Max modcod: 13 


In [361]:
util.plot_used_passes(case, instances_df, problem, res.X[ind_close,:])
plt.gca().set_position(bbox)
plt.gca().yaxis.labelpad = -3.0

plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%srange_selection.pdf' % prefix)

In [362]:
util.plot_pointing(case, problem, res.X[ind_close,:])
plt.gca().set_position(bbox)
plt.gca().yaxis.labelpad = 2.0

plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%spointing.pdf' % prefix)

In [363]:
hdf_store.close()